In [2]:
# +============================================================================+
# | Company:   SOFiSTiK AG                                                     |
# | Version:   SOFiSTIK 2022                                                   |
# +============================================================================+
from sofistik_daten import *
import os               # for the environment variable necessary, this is a great tool
import platform         # checks the python platform
import string
from ctypes import *    # read the functions from the cdb
import math
import pandas as pd

In [3]:
# This example has been tested with Python 3.7 (64-bit)
##print ("The path variable=", os.environ["Path"])

# Check the python platform (32bit or 64bit)
##print ("Python architecture=", platform.architecture())
sofPlatform = str(platform.architecture())



# Set DLL dir path - new in PY 3.8 for ctypes
# See: https://docs.python.org/3/whatsnew/3.8.html#ctypes
os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2022\SOFiSTiK 2022\interfaces\64bit")
os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2022\SOFiSTiK 2022")

# Get the DLL functions
myDLL = cdll.LoadLibrary(r"C:\Program Files\SOFiSTiK\2022\SOFiSTiK 2022\sof_cdb_w-2022.dll")
py_sof_cdb_get = cdll.LoadLibrary("sof_cdb_w-2022.dll").sof_cdb_get
py_sof_cdb_get.restype = c_int

py_sof_cdb_kenq = cdll.LoadLibrary("sof_cdb_w-2022.dll").sof_cdb_kenq_ex


# Connect to CDB
Index = c_int()
cdbIndex = 99

# input the cdb path here
fileName = r"C:\Users\nooyur\Documents\01 Arbeidsmappe\01 Prosjektmappe\03 Betna - Hestnes\Beregninger\Sofistik\Hestnessletta_V5\Hestnessletta.cdb"

# important: Unicode call!
Index.value = myDLL.sof_cdb_init(fileName.encode('utf8'), cdbIndex)

# get the CDB status
cdbStat = c_int()
cdbStat.value = myDLL.sof_cdb_status(Index.value)

# Print the Status of the CDB
print ("CDB Status:", cdbStat.value)

pos = c_int(0)
datalen = c_int(0)

a = c_int
ie = c_int()

datalen.value = sizeof(CNODE)
RecLen = c_int(sizeof(cnode))

"""
do while ie == 0, see cdbase.chm, Returnvalue.
   = 0 -> No error
   = 1 -> Item is longer than Data
   = 2 -> End of file reached
   = 3 -> Key does not exist
"""
"""
while ie.value < 2:
   ie.value = py_sof_cdb_get(Index, 20, 0, byref(cnode), byref(RecLen), 1)
   print("{:10d}{:10d}{:10d}{:10d}{:10.2f}{:10.2f}{:10.2f}".format(
      cnode.m_nr,      # node-number
      cnode.m_inr,     # internal node-number
      cnode.m_kfix,    # degree of freedoms
      cnode.m_ncod,    # additional bit code
      cnode.m_xyz[0],  # x coordinates
      cnode.m_xyz[1],  # y coordinates
      cnode.m_xyz[2])  # z coordinates
   ) 

loadcase = 1

forces = []
RecLen = c_int(sizeof(cbeam_foc))
while ie.value == 0:
   ie.value = py_sof_cdb_get(Index, 102, loadcase, byref(cbeam_foc), byref(RecLen), 1) 
   #if cbeam_foc.m_id == 200007:
   #   print(cbeam_foc.m_n)
   if 200001 <= cbeam_foc.m_id <= 200007: 
      forces.append(math.ceil(cbeam_foc.m_n))
      #print(math.ceil(cbeam_foc.m_n))
   
RecLen = c_int(sizeof(cbeam_foc))
myDLL.sof_cdb_flush(Index.value)
#print(forces)
"""


datalen.value = sizeof(CNODE)
RecLen = c_int(sizeof(cnode))

#print(datalen.value)



CDB Status: 3


In [4]:
def GetNodesPosition(cdb_link):
    # Connect to CDB
    Index = c_int()
    cdbIndex = 99

    # input the cdb path here
    fileName = cdb_link

    # important: Unicode call!
    Index.value = myDLL.sof_cdb_init(fileName.encode('utf8'), cdbIndex)

    # get the CDB status
    cdbStat = c_int()
    cdbStat.value = myDLL.sof_cdb_status(Index.value)

    # Print the Status of the CDB
    #print ("CDB Status:", cdbStat.value)

    pos = c_int(0)
    datalen = c_int(0)

    a = c_int
    ie = c_int()

    datalen.value = sizeof(CNODE)
    RecLen = c_int(sizeof(cnode))
    

    NodeNum = []
    IntNodeNum = []
    X_Coor = []
    Y_Coor = []
    Z_Coor = []
    
    
    
    while ie.value < 2:
        ie.value = py_sof_cdb_get(Index, 20, 0, byref(cnode), byref(RecLen), 1)
        #if cnode.m_nr in nodeFilter:
        NodeNum.append(cnode.m_nr)
        IntNodeNum.append(cnode.m_inr)
        X_Coor.append((round(cnode.m_xyz[0],3)))
        Y_Coor.append((round(cnode.m_xyz[1],3)))
        Z_Coor.append((round(cnode.m_xyz[2],3)))
    
        Data = {'NodeNum' : NodeNum, 'IntNodeNum' : IntNodeNum,'X_Coor' : X_Coor,'Y_Coor' : Y_Coor,'Z_Coor' : Z_Coor}
        
        RecLen = c_int(sizeof(cnode))
    myDLL.sof_cdb_close(0)
    cdbStat.value = myDLL.sof_cdb_status(Index.value)
    
    return pd.DataFrame(Data)
    #return NodeNum
    

In [5]:
#cdb_link = r"C:\Users\nooyur\Documents\01 Arbeidsmappe\01 Prosjektmappe\03 Betna - Hestnes\Beregninger\Sofistik\Hestnessletta_V5\Hestnessletta.cdb"
#cdb_link = r"C:\Users\nooyur\Documents\01 Arbeidsmappe\Sofistik\2D Bjelke.cdb"
cdb_link = r"C:\Users\nooyur\Documents\01 Arbeidsmappe\Sofistik\CDB\CDB.cdb"
lst = [0,1]
nodeData = GetNodesPosition(cdb_link)
nodeData

,NodeNum,IntNodeNum,X_Coor,Y_Coor,Z_Coor
0,1,1,0.00,0.0,0.0
1,2,2,10.00,0.0,0.0
2,1000,3,1.25,0.0,0.0
3,1001,4,2.50,0.0,0.0
4,1002,5,3.75,0.0,0.0
5,1003,6,5.00,0.0,0.0
6,1004,7,6.25,0.0,0.0
7,1005,8,7.50,0.0,0.0
8,1006,9,8.75,0.0,0.0
9,1006,9,8.75,0.0,0.0


In [35]:
def GetBeamForces(cdb_link):
    # Connect to CDB
    Index = c_int()
    cdbIndex = 99

    # input the cdb path here
    fileName = cdb_link

    # important: Unicode call!
    Index.value = myDLL.sof_cdb_init(fileName.encode('utf8'), cdbIndex)

    # get the CDB status
    cdbStat = c_int()
    cdbStat.value = myDLL.sof_cdb_status(Index.value)

    # Print the Status of the CDB
    #print ("CDB Status:", cdbStat.value)

    pos = c_int(0)
    datalen = c_int(0)

    a = c_int
    ie = c_int()

    datalen.value = sizeof(CBEAM_FOC)
    RecLen = c_int(sizeof(cbeam_foc))
    

    beamNum = []
    beamCoord = []
    my = []
    vz = []
    
    
    
    
    while ie.value < 2:
        ie.value = py_sof_cdb_get(Index, 102, 1, byref(cbeam_foc), byref(RecLen), 1)
        beamNum.append(cbeam_foc.m_id)
        beamCoord.append(cbeam_foc.m_x)  
        my.append((round(cbeam_foc.m_my,3)))
        my.append((round(cbeam_foc.m_vz,3)))
        #Data = {'BeamnNum' : beamNum, 'my' : my,'vz' : vz}
       
        #RecLen = c_int(sizeof(cbeam_foc))
    myDLL.sof_cdb_close(0)
    cdbStat.value = myDLL.sof_cdb_status(Index.value)
    
    #return pd.DataFrame(Data)
    return beamNum,beamCoord,my,vz

In [50]:
cdb_link = r"C:\Users\nooyur\Documents\01 Arbeidsmappe\Sofistik\CDB\CDB.cdb"
beamforces = GetBeamForces(cdb_link)
noder = beamforces[0]
nodeCoord= beamforces[1]
krefter= beamforces[2]
l = len(noder)
l

krefter_delt = list(chunks(krefter,l))
my = krefter_delt[0]

vz = krefter_delt[1]

xval = []

for i in range(0,len(nodeCoord):
    xval.append(NodeCoord[i+1]
    

Data = {'Noder' : noder,'NodeCoord' : nodeCoord, 'My' : my,'Vz' : vz}
pd.DataFrame(Data)


,Noder,NodeCoord,My,Vz
0,0,1.25,56.422,9.954
1,0,0.00,49.954,50.229
2,10001,0.00,-99.541,9.954
3,10001,1.25,-30.046,56.422
4,10002,0.00,-99.541,-0.046
5,10002,1.25,49.954,56.422
6,10003,0.00,-43.349,-0.046
7,10003,1.25,39.954,50.115
8,10004,0.00,-43.349,-10.046
9,10004,1.25,39.954,50.115


In [40]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [61]:
xval = []
xval = [nodeCoord[1]]
for i in range(0,len(nodeCoord)-1):
    xval.append(nodeCoord[i]+xval[-1])
                
nodeCoord

[1.25,
 0.0,
 0.0,
 1.25,
 0.0,
 1.25,
 0.0,
 1.25,
 0.0,
 1.25,
 0.0,
 1.25,
 0.0,
 1.25,
 0.0,
 1.25,
 0.0,
 1.25,
 1.25,
 2.5,
 2.5,
 2.5,
 3.75,
 3.75,
 5.0,
 5.0,
 6.25,
 6.25,
 7.5,
 7.5,
 8.75,
 8.75,
 10.0,
 10.0,
 11.25,
 11.25,
 12.5]